<a href="https://colab.research.google.com/github/mkane968/Extracted-Features/blob/master/From_Texts_to_Topic_Models_A_Complete_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Use this code to convert any corpus of plain text files to disaggregated data and then use that data to create LDA topic models. 

#Install Packages

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
!pip3 install spacy
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import io
import re
from pprint import pprint

#Part 1: Text Sectioning and Disaggregation

Use this code to clean, section, and disaggregate texts and corpora. 

**Why Perform Text Sectioning?** 

Dividing texts into sections (for example, chapters or chunks of N length) is valuable as a precursor to topic modeling and other forms of computational analysis which perform more accurately when applied to groups of segmented documents from longer texts. 

**Why Disaggregate Texts?** 

The process of disaggregating the words in texts (in this case, by alphabetizing them) also creates data sets that can be shared freely where original texts cannot be due to copyright restrictions. 

*Input/Output Specifications:* 

This code requires plain txt files as input, either those from this repository's sample_data folder or those from a local machine. It returns csv files with disaggregated text grouped by chapter or chunk of n length.

##Upload and Add Text Files To Pandas DataFrame
In this section, text files are mounted to Google Drive from the local machine and then loaded into a Pandas DataFrame. Pandas is a fast and relatively easy way to work with large datasets. Though data frames are typically associated with numbers, Pandas also offers many functionalities for [working with textual data. ](https://www.tutorialspoint.com/python_pandas/python_pandas_working_with_text_data.htm) 

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Selet all files to upload
from google.colab import files

uploaded = files.upload()

In [ ]:
#Add files into dataframe
import pandas as pd

books = pd.DataFrame.from_dict(uploaded, orient='index')
books

In [ ]:
#Reset index and add column names to make wrangling easier
books = books.reset_index()
books.columns = ["Title", "Text"]
books

In [ ]:
#Remove encoding characters from Text column (b'\xef\xbb\xbf)
books['Text'] = books['Text'].apply(lambda x: x.decode('utf-8'))

In [ ]:
#Change data type to string (to support further cleaning below)
books = books.astype(str)

##Clean Texts and Set Parameters for Sectioning
Several basic cleaning processes are implemented: removing unwanted characters from titles, removing newline characters from texts, and removing punctuation. Parameters are also set for part(s) of text to be included in sectioning. In the SciFi Corpus project, "START OF BOOK" and "END OF BOOK" tags were added to delineate the body of each text. Code in this section removes any text outside the starting and ending parameters--e.g., title page, copyright page, other paratext. 

In [ ]:
#Remove .txt from titles
books['Title'] = books['Title'].str.replace(r'.txt', ' ', regex=True) 
books

In [ ]:
#Remove newline characters
books['Text'] = books['Text'].str.replace(r'\s+|\\r', ' ', regex=True) 
books['Text'] = books['Text'].str.replace(r'\s+|\\n', ' ', regex=True) 
books

In [ ]:
#Remove punctuation
books['Text'] = books['Text'].str.replace(r'[^\w\s]+', '', regex = True)

In [ ]:
#Remove paratext (before and after START OF BOOK and END OF BOOK tags)
#If texts you are working with do not have these tags, ignore this cell

#Split book on start of book tag, keep text only after start of book tag
start = books["Text"].str.split("START OF BOOK", expand = True)
books['Text'] = start[1]

#Split book on end of book tag, keep text only before of book tag
end = books["Text"].str.split("END OF BOOK", expand = True)
books['Text'] = end[0]
books

In [ ]:
#Check that text is cleaned and sectioned
books.iloc[0]['Text']

In [ ]:
#Define new dataframe
books_cleaned = books

##Section Texts By Chapter Headings
When working with texts with clearly delineated chapters, using chapter headings is a relatively easy way to section texts into segments of (relatively) the same size. After checking the chapter counts for each text to confirm whether sectioning by chapter is a useful procedure, this code iterates through the texts and splits them each time it encounters a new "chapter" heading. From here, the text from each chapter is appended to a new dataframe and denoted by book and chapter number. 

In [ ]:
#Count number of chapters in each text
chapter_counts = books_cleaned['Text'].str.count('CHAPTER')

#Append chapter counts to dataframe
books_cleaned["Chapters"] = chapter_counts
books_cleaned

In [ ]:
#Make new cell each time new chapter starts 
new = books_cleaned["Text"].str.split("CHAPTER", expand = True).set_index(books_cleaned['Title'])
new

In [ ]:
#Flatten dataframe so each chapter is on own row, designated by book and chapter 
chapters_df = new.stack().reset_index()
chapters_df.columns = ["Book", "Chapter", "Text"]
chapters_df

In [ ]:
#Tidying the DF
#Combine book and chapter labels into one column
chapters_df['Book + Chapter'] = chapters_df['Book'].astype(str) + '_Chapter_' + chapters_df['Chapter'].astype(str)

#Remove individual book and chapter columns
chapters_df.drop(columns=['Book', 'Chapter'])

#Reindex so book + chapter is first column 
column_names = "Book + Chapter", "Text"
chapters_df = chapters_df.reindex(columns=column_names)
chapters_df

##Section Texts By Chunk of N Length
When working with texts WITHOUT discernable chapter headings--or, even if chapter headings are present but too infrequent to split texts into meaningful segments--texts can instead be sectioned by chunks of "N" length, where N is a variable that can be custom-set below. After checking the word counts for each text to determine what size chunks would be appropriate, this code iterates through the texts and splits them each time it counts "N" number of words. From here, the text from each chunk is appended to a new dataframe and denoted by book and chunk number.

In [ ]:
#Get number of words in each book (helps to determine chunk length)
words = books_cleaned["Text"].apply(lambda x: len(str(x).split(' ')))

#Append word counts to dataframe
books_cleaned["Word Count"] = words
books_cleaned

In [ ]:
#Tokenize Text
import nltk
nltk.download('punkt')
books_cleaned['Tokens'] = books_cleaned.apply(lambda row: nltk.word_tokenize(row['Text']), axis=1)
books_cleaned

In [ ]:
#Define chunking function
def split(list_a, chunk_size):
  for i in range(0, len(list_a), chunk_size):
    yield list_a[i:i + chunk_size]

#Set desired size of chunks
chunk_size = 1000

#Create new list for chunked sentences
chunked_sentences = []

#Perform chunking function on each row of tokens
s = books_cleaned['Tokens']
for content in s:
  chunks = list(split(content, chunk_size))
  #Check that text is being chunked correctly
  print(chunks[0])
  #Convert from tokens to sentences to add to new df
  for chunk in chunks:
    sentences = ' '.join(chunk)
    chunked_sentences.append(sentences)

In [ ]:
#Create new dataframe to store chunked sentences for disaggregation
#NEEDS WORK--"Title" does not reflect which chunks belong to which books
df = pd.DataFrame(chunked_sentences).T
chunked_df = df.stack().reset_index()
chunked_df.columns = ["Title","Chunk","Text"]
chunked_df

In [ ]:
#Tidying the DF
#Combine book and chunk labels into one column
chunked_df['Book + Chunk'] = chunked_df['Title'].astype(str) + ' Chunk ' + chunked_df['Chunk'].astype(str)

#Remove individual book and chunk columns
chunked_df.drop(columns=['Title', 'Chunk'])

#Reindex so book + chunk is first column 
column_names = "Book + Chunk", "Text"
chunked_df = chunked_df.reindex(columns=column_names)
chunked_df

##Disaggregate Texts and Download CSV Output
Working with texts split by chapter or chunk, the final step of this process is to disaggregate the data. Disaggregation, or the breakdown of data into smaller (disordered) parts, is accomplished through the alphabetization of the words in each chapter/chunk. 

The resulting "bag of words" data can then be downloaded as csvs and used for further analysis, such as through the Topic Modeling pipeline in the Extracted Features repository: https://github.com/SF-Nexus/Extracted-Features/blob/main/Topic%20Modeling%20with%20SciFi%20Corpus.ipynb 

In [ ]:
#Working with data from texts sectioned by CHAPTER
#Alphabetize words in each chapter string
chapters_df['Text'] = chapters_df['Text'].apply(lambda x: ' '.join(sorted(x.split())))
chapters_df

In [ ]:
#Download disaggregated chapters to csv
from google.colab import files

chapters_df.to_csv('sci-fi-chapters_bag_of_words_output.csv', encoding = 'utf-8-sig') 
files.download('sci-fi-chapters_bag_of_words_output.csv')

In [ ]:
#Working with data from texts sectioned by CHUNK of N length
#Alphabetize words in each chunk string
chunked_df['Text'] = chunked_df['Text'].apply(lambda x: ' '.join(sorted(x.split())))
chunked_df

In [ ]:
#Download disaggregated chunks to csv
from google.colab import files

chunked_df.to_csv('sci-fi_word_chunks_bag_of_words_output.csv', encoding = 'utf-8-sig') 
files.download('sci-fi_word_chunks_bag_of_words_output.csv')

#Part 2: Creating Topic Models
What are the most common topics in a corpus of science fiction texts? Which texts tend to address the same topics, and why? These are just a couple of the many questions which this code addresses. Below, Gensim is used to create LDA topic models of a pre-loaded dataframe of texts. 

This code is adapted from this [Intro to Topic Modeling with Gensim and pyLDAvis](https://github.com/hawc2/text-analysis-with-python/blob/master/Topic_Modeling.ipynb) and works well with input from from the Text Sectioning and Disaggregation code from [this repository](https://github.com/SF-Nexus/Extracted-Features/blob/main/Text_Sectioning_and_Disaggregation_in_Python.ipynb)

Topic modeling resources: 
https://maria-antoniak.github.io/2022/07/27/topic-modeling-for-the-people.html

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#13viewthetopicsinldamodel

https://github.com/polsci/colab-gensim-mallet/blob/master/topic-modeling-with-colab-gensim-mallet.ipynb

http://www.cs.columbia.edu/~blei/papers/Blei2011.pdf

https://github.com/laurejt/authorless-tms 

https://maria-antoniak.github.io/resources/2019_cscw_birth_stories.pdf

Data display: https://colab.research.google.com/notebooks/data_table.ipynb#scrollTo=jcQEX_3vHOUz 

##Convert Disaggregated Data into New DF and Clean

In [ ]:
#Convert csv to dataframe
#Copy name of csv created above with chunked or chapterized data 
df = pd.read_csv(io.StringIO(uploaded['sci-fi_word_chunks_bag_of_words_output.csv.csv'].decode('utf-8')))
df

In [ ]:
#Drop any empty cells
df = df.dropna()
df

In [ ]:
df['Book + Chapter']

In [ ]:
#Add values in Text column to new list (for further cleaning)
data = df.Text.values.tolist()

In [ ]:
#View dataframe as Colab data table
%load_ext google.colab.data_table 
df

##Clean Texts

In [ ]:
#Define list of stopwords
stop_words = stopwords.words('english')

# Add further stopwords by simply "appending" desired words to dictionary
#stop_words.append('movie')

In [ ]:
#Remove punctuation
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

In [ ]:
#Define function to perform simple preprocessing on text
def sent_to_words(sentences):
    for sentence in sentences:
      yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [ ]:
#Run processing function on texts
data_words = list(sent_to_words(data))
print(data_words[:10])

In [ ]:
#Define bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
#Define stopword removal
def remove_stopwords(texts):
   return [[word for word in simple_preprocess(str(doc))
if word not in stop_words] for doc in texts]

#Define function to make bigrams
def make_bigrams(texts):
   return [bigram_mod[doc] for doc in texts]

#def make_trigrams(texts):
#   return [trigram_mod[bigram_mod[doc]] for doc in texts]

#Define function to lemmatize texts
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
   texts_out = []
   for sent in texts:
     doc = nlp(" ".join(sent))
     texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
   return texts_out

In [ ]:
#Run functions to remove stopwords, make bigrams, and lemmatize text
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=[
   'NOUN', 'ADJ', 'VERB', 'ADV'
])
print(data_lemmatized[:4])

##Building Dictionary and Corpus

In [ ]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus)

##Create Topic Model - Topics 20

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=2,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
##View Keywords of Each Topic in the Model
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
#Calculate Complexity and Coherence Score
#These measures help determine how good the model is: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#13viewthetopicsinldamodel

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
#Create Visualization (Save HTML)
#The easiest way to create the visualization is to reveal it in the Google Colab notebook and save it as an html file that you can view on your browser. 

vis = gensimvis.prepare(lda_model, corpus, id2word)

#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')

In [ ]:
pyLDAvis.save_html(vis, '/content/LDAviz.html')

In [ ]:
pyLDAvis.display(vis)

##Find the Optimal Number of Topics

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

## Topic Modeling Model - 15 Topics

In [ ]:
lda_model15 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=2,
                                           chunksize=100,
                                           passes=20,
                                           iterations=200,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Create Visualization (Save HTML)

#The easiest way to create the visualization is to reveal it in the Google Colab notebook and save it as an html file that you can view on your browser. 

vis15 = gensimvis.prepare(lda_model15, corpus, id2word)
#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')

In [ ]:
pyLDAvis.save_html(vis60, '/content/LDAviz15.html')

In [ ]:
pyLDAvis.display(vis15)

##Compare: LDA Mallet Model

In [ ]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip
mallet_path = '/content/mallet-2.0.8/bin/mallet' 

In [ ]:
#Get mallet path and run new topic model
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

In [ ]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
#Create visualization
converted_ = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
vis_mallet = gensimvis.prepare(converted_, corpus, id2word)

In [ ]:
#Display visualization 
pyLDAvis.display(vis_mallet)

In [ ]:
#Get ideal number of topics
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

##Look at Topics Per Document

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

## Serve Visualization in Browser

You can also serve the visualization locally in the browser using the below chunk of code. Beware that caching in your browser and other issues, such as ad-blockers, may require some debugging to get this working on your machine. 

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.show(vis)